In [1]:
import numpy as np
import scipy.stats as stats
import scipy.linalg as linalg
import scipy.optimize as opt
import scipy.sparse as sparse
import matplotlib.pyplot as plt

Введём параметры нашей системы. Мы будем рассматривать вектора размерности $n$, в которых не более $k$ ненулевых компонент. Мы будем отображать эти вектора в $m$-мерное пространство. $m$ я прикинул из статьи как $\sim k \log_2 \frac{n}{k}$.

In [3]:
n, m, k = 100, 30, 7
a = stats.norm.rvs(size=(m, n))
a = stats.bernoulli.rvs(p=1/2, size=(m, n))

Следующая функция генерирует разреженные вектора:

In [9]:
def gen(n, k):
    x = np.random.rand(n)
    random_mask = np.concatenate((np.zeros(n - k), np.ones(k)))
    np.random.shuffle(random_mask)
    return x * random_mask

In [2]:
def gen():
    return stats.pareto.rvs(1, size=n)

Сжатие --- это просто домножение на матрицу

In [4]:
def compress(x):
    return a @ x

Я ищу решение в линейном пространстве решений системы $y = Ax$, минимизируя $L_1$-норму $x$.

In [58]:
def decompress0(y):
    x0 = linalg.lstsq(a, y)[0]
    linspace = linalg.null_space(a)
    if linspace.size == 0:
        return x0
    
    l = linspace
    fun = lambda c: linalg.norm(l @ c + x0, ord=1)
    res = opt.minimize(fun=fun, x0=np.zeros(n - m))
    return linspace @ res.x + x0

In [163]:
linspace = linalg.null_space(a)
def decompress1(y):
    x0 = linalg.lstsq(a, y)[0]
    if linspace.size == 0:
        return x0
    
    l = linspace
    n, m = l.shape
    A_ub = np.zeros((n * 2, m + n))
    A_ub[0 : n, 0 : m] = l
    A_ub[0 : n, m : m + n] = -np.identity(n)
    A_ub[n : 2 * n, 0 : m] = -l
    A_ub[n : 2 * n, m : m + n] = -np.identity(n)
    
    b_ub = np.zeros(2 * n)
    b_ub[0 : n] = -x0
    b_ub[n : 2 * n] = x0
    
    c = np.zeros(m + n)
    c[m : m + n] = 1
    
    bounds=np.array([(None, None)] * m + [(0, None)] * n)
    
    res = opt.linprog(c, A_ub=A_ub, b_ub=b_ub, bounds=bounds, 
                      #options={'maxiter': 10000, 'disp': False, 'tol': 1e-12, 'bland': False})
                      method='interior-point')
    if not res.success:
        print("Failure!")
        raise Exception(res)
    return linspace @ res.x[0 : m] + x0
    

In [5]:
def decompress(y):
    m, n = a.shape
    A_eq = np.zeros((m, 2 * n))
    A_eq[:, :n] = a

    A_ub = np.zeros((2 * n, 2 * n))
    A_ub[:n, :n] = np.identity(n)
    A_ub[:n, n:] = -np.identity(n)
    A_ub[n:, :n] = -np.identity(n)
    A_ub[n:, n:] = -np.identity(n)
    #A_ub = sparse.csr_matrix((2 * n, 2 * n))
    #A_ub.setdiag(np.concatenate((np.ones(n), -np.ones(n))))
    #A_ub.setdiag(-np.ones(n), k=n)
    #A_ub.setdiag(-np.ones(n), k=-n)
    b_ub = np.zeros(2 * n)
    bounds=np.array([(None, None)] * n + [(0, None)] * n)
    c = np.zeros(2 * n)
    c[n:] = 1
    b_eq = y
    res = opt.linprog(c, A_ub=A_ub, b_ub=b_ub, A_eq=A_eq, b_eq=b_eq, bounds=bounds, 
                      #options={'maxiter': 10000, 'disp': False, 'tol': 1e-12, 'bland': False})
                      method='interior-point')
    if not res.success:
        print("Failure!")
        raise Exception(res)
    return res.x[0 : n]
    

In [6]:
import time

Посмотрим на количество ошибок в декодировании:

In [10]:
t = 0
failures = 0
for i in range(100):
    x = gen()
    y = compress(x)
    t0 = time.process_time()
    try:
        res = decompress(y)
        t1 = time.process_time()
        t += t1 - t0
        failures += linalg.norm(res - x)
    except:
        pass
print(failures)
print(t)

/usr/local/lib/python3.6/dist-packages/scipy/optimize/_linprog_ip.py:1262: OptimizeWarning: Solving system with option 'sym_pos':True failed. It is normal for this to happen occasionally, especially as the solution is approached. However, if you see this frequently, consider setting option 'sym_pos' to False.
  OptimizeWarning)
/usr/local/lib/python3.6/dist-packages/scipy/optimize/_linprog_ip.py:1274: OptimizeWarning: Solving system with option 'sym_pos':False failed. This may happen occasionally, especially as the solution is approached. However, if you see this frequently, your problem may be numerically challenging. If you cannot improve the formulation, consider setting 'lstsq' to True. Consider also setting `presolve` to True, if it is not already.
  OptimizeWarning)


2.8839235655188916e-07
55.350424165999925


In [150]:
t = 0
failures = 0
for i in range(10):
    x = gen()
    y = compress(x)
    t0 = time.process_time()
    res = decompress(y)
    t1 = time.process_time()
    t += t1 - t0
    res1 = res.copy()
    res1 -= x
    res1[res1 < 1e-5] = 0
    if np.linalg.norm(res1, ord=0) > 0:
        failures += 1
print(failures)
print(t)

10
4.952107003000947


Пока не слишком удачный результат. Попробуем другое распределение.

In [142]:
a = stats.bernoulli.rvs(p=1/2, size=(m, n))

In [148]:
failures = 0
for i in range(100):
    x = gen()
    y = compress(x)
    res = decompress(y)
    res1 = res.copy()
    res1 -= x
    res1[res1 < 1e-5] = 0
    if np.linalg.norm(res1, ord=0) > 0:
        failures += 1
print(failures)

Failure!


Exception:      con: array([4.18787785e-01, 4.18758648e-01, 4.18771251e-01, 4.18679553e-01,
       3.11667855e-04, 4.18747231e-01, 4.23505633e-04, 4.82595788e-04,
       4.18700255e-01, 4.18791379e-01, 5.02968419e-04, 5.20698114e-04,
       4.18758964e-01, 2.83490154e-04, 3.76910867e-04, 4.18822062e-01,
       4.18775775e-01, 4.18819890e-01, 4.90792596e-04, 4.18782827e-01,
       4.18730560e-01, 4.46932185e-04, 4.18785244e-01, 3.71368771e-04,
       3.83261682e-04, 4.18831018e-01, 4.18701446e-01, 4.18805284e-01,
       4.18766698e-01, 4.18825286e-01])
     fun: 300612.2773355461
 message: 'The solution does not satisfy the constraints, yet no errors were raised and there is no certificate of infeasibility or unboundedness. This is known to occur if the `presolve` option is False and the problem is infeasible. If you uncounter this under different circumstances, please submit a bug report. Otherwise, please enable presolve.'
     nit: 12
   slack: array([6.78408759e-03, 5.85750757e-03, 

Гораздо лучше! Но всё равно, я полагаю, дело в том, что я выбрал $m$ недостаточно большим, а если брать его ещё больше, то оно станет совсем уж близко к $n$. Нужно научиться быстрее оптимизировать $L_1$-норму, потому что иначе это работает слишком медленно.